In [ ]:
pip install requests beautifulsoup4


In [ ]:
pip install mysql-connector-python


In [ ]:
import requests
from bs4 import BeautifulSoup
import mysql.connector
import time


url = 'https://books.toscrape.com/catalogue/page-{}.html'

def get_page_content(page_url):
    response = requests.get(page_url)
    return BeautifulSoup(response.content, 'html.parser') 


def scrape_books_from_page(page_url):
    soup = get_page_content(page_url)
    book_containers = soup.find_all('article', class_='product_pod')
    books = []
    for container in book_containers:
        books.append(scrape_book_details(container))
    return books

def scrape_book_details(book_container):
    Title = book_container.find('h3').find('a')['title']
    Cost = book_container.find('p', class_='price_color').text.strip('£').replace('Â', '')
    Rating_text = book_container.find('p', class_='star-rating')['class'][1]
    Stock_Availability = book_container.find('p', class_='instock availability').text.strip()

    # Map rating text → numeric value
    rating_mapping = {
        "One": 1.0,
        "Two": 2.0,
        "Three": 3.0,
        "Four": 4.0,
        "Five": 5.0
    }
    Rating = rating_mapping.get(Rating_text, 0.0)

    return Title, Cost, Rating, Stock_Availability

    
page_url = url.format(1)


books_on_first_page = scrape_books_from_page(page_url)

for book in books_on_first_page:
    print(f"Title: {book[0]}, Cost: £{book[1]}, Rating: {book[2]} Stock_Availability: {book[3]}")




In [ ]:
all_books = []

page_number = 1

while True:
    # Construct the URL for the current page
    page_url = url.format(page_number)
    
    # Scrape books from the current page
    full_book_and_containers_on_page = scrape_books_from_page(page_url)

    # If no books are found, break the loop (assuming no books means we've reached the end)
    if not full_book_and_containers_on_page:
        break
    
    # Add the books from the current page to the all_books list
    all_books.extend(full_book_and_containers_on_page)

    print(f"Page {page_number}: Found {len(full_book_and_containers_on_page)} books")
    
    page_number += 1
time.sleep(2)

for book in all_books:
    Title, Cost, Rating, Stock_Availability = book
    savetoDB(Title, Cost, Rating, Stock_Availability)
